# Data Analysis 3 Prediction: Assignment 1
Looking at the occupation of Legal Professions: Lawyers, Juges, Magistrates, and other judicial positions (occ2012 = 2100)

Creating 4 linear regression models with each new model introducing more complexity, then comparing their measures of fit and performance

https://osf.io/uqe8z = description for what each variable means

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
from plotnine import *
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from statsmodels.tools.eval_measures import mse,rmse
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore")

### Reading Data & Exploratory Data Analysis

In [2]:
#reading file. morg2014.csv was too large- had to turn into zip file
file = pd.read_csv('ezyzip.zip')

In [3]:
#see data
file.shape

(317056, 96)

In [4]:
#Legal Professions like lawyers, judges, magistrates, and other judicial positions
#at start: 1,666 observations
law = file.loc[file['occ2012'] == 2100].reset_index(drop=True)
print(law.shape)
law.head()

(1666, 96)


,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
0,960021970039060,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,18.0,Legal occupations,Professional and Technical services
1,235004820400137,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,86,...,648,633,0,0,0,0,0,18.0,Legal occupations,Public Administration
2,985879730586005,January,3.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,71,...,648,633,0,0,0,1,0,18.0,Legal occupations,Professional and Technical services
3,94927107035608,January,2.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,93,...,648,645,0,0,1,0,0,18.0,Legal occupations,Professional and Technical services
4,97977400095618,January,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,93,...,648,645,0,0,0,1,0,18.0,Legal occupations,Professional and Technical services


### Plan of Action

 y = earnings per hour ('earnperhr'). this means... earnwke (weekly earnings)/uhourse(usual hours worked)
    
 Model 1: age, sex(female dummy)
 
 Model 2: age, sex(female dummy), uhourse, grade92(ed_low/high dummy)
 
 Model 3: age, sex(female dummy), uhourse, grade92(ed_low/high dummy), state division (dummy), race(dummy), prcitshp(dummy),
 
 Model 4: age, sex(female dummy), uhourse, grade92(ed_low/high dummy), state division(dummy), race(dummy), prcitshp(dummy), marital(dummy), ownchild(dummy)

- could do 'state' as a variable, but would have to create dummy variables... (str)
- could do 'race' as a variable, but would have to create dummy variables... (int)
- could do 'prcitshp' (citizenship status) as a variable, but would have to create dummy variable...(str)
- could do 'marital' as variable...dummy variables would be 1,2,3 = 'married', 4,5,6,7 = 'single'(int)
- could do 'ownchild' as variable = shows number of children less than 18 in the primary family(int)

- deleting ethnic category because only 68/1,666 were not null!

**I am choosing these predictors as I feel that each could possibly have a large role to play in what determines the pay (earnings per week) of those in law professions.**

### Variable Legend
- age = age of employee in years
- sex = 1 if male, 2 if female
- uhourse = usual hours worked per week
- grade92 = education level (ints 31-46 representing Kindergarten-doctorate degree)
- earnwke = weekly earnings in USD
- state = in which state the employee resides
- race = 1:White, 2:Black, 3:American Indian, 4:Asian or Pacific Islander, 5:Other, 6:Mixed(White/Black), 7:Mixed(White/American Indian), 8:Mixed(White/Asian), 10:Mixed(Black/American Indian), 11:Mixed(Black/Asian)
- prcitshp = 1 if American citizen, 0 if non-American citizen
- marital = 1-3 if married, 4-7 if currently single
- ownchild = number of children employee has in primary family

### Data Work, Altering Variables

In [5]:
#dropping earnwke and uhourse variables with nan values because need those to compute y and cannot predict y 
#properly if the values were imputed!!

#dropping observations where values for variable 'earnwke' are nan
law.earnwke.isnull().value_counts()
law.dropna(subset=['earnwke'], inplace=True)
#check to make sure only non-null values
law.earnwke.isnull().value_counts()

#dropping observations where values for variable 'uhourse' are nan
law.uhourse.isnull().value_counts()
law.dropna(subset=['uhourse'], inplace=True)
#check to make sure only non-null values
law.earnwke.isnull().value_counts()

#There are now 1,119 observations in the law dataframe

earnwke
False    1119
Name: count, dtype: int64

In [6]:
#checking for null values:

#age: no missing values
law.age.isnull().value_counts()

#sex: no missing values
law.sex.isnull().value_counts()

#grade92: no missing values
law.grade92.isnull().value_counts()

#state: no missing values
law.state.isnull().value_counts()
#USING DIVISIONS instead

#race: no missing values
law.race.isnull().value_counts()

#prcitshp: no missing values
law.prcitshp.isnull().value_counts()
#USING dummy variable instead of str

#marital: no missing values
law.marital.isnull().value_counts()
#USING married and single dummy variables

#ownchild: no missing values
law.ownchild.isnull().value_counts()


ownchild
False    1119
Name: count, dtype: int64

In [7]:
#creation of outcome variable (y)
law['earnperhr'] = law['earnwke']/law['uhourse']
law['earnperhr'][:5]

0    28.84600
1    34.61525
4    46.14000
6    14.00000
7    57.69220
Name: earnperhr, dtype: float64

In [8]:
#fixing state variable (some were int and some were str), making into divisions
#print(law.state.unique())

#NE Division
law.state.replace('MA', 11, inplace = True)
law.state.replace('NH', 12, inplace = True)
law.state.replace('VT', 13, inplace = True)
law.state.replace('MA', 14, inplace = True)
law.state.replace('RI', 15, inplace = True)
law.state.replace('CT', 16, inplace = True)
#Middle Atlantic Division
law.state.replace('NY', 21, inplace = True)
law.state.replace('NJ', 22, inplace = True)
law.state.replace('PA', 23, inplace = True)
#East North Central Division
law.state.replace('OH', 31, inplace = True)
law.state.replace('IN', 32, inplace = True)
law.state.replace('IL', 33, inplace = True)
law.state.replace('MI', 34, inplace = True)
law.state.replace('WI', 35, inplace = True)
#West North Central Division
law.state.replace('MN', 41, inplace = True)
law.state.replace('IA', 42, inplace = True)
law.state.replace('MO', 43, inplace = True)
law.state.replace('ND', 44, inplace = True)
law.state.replace('SD', 45, inplace = True)
law.state.replace('46', 46, inplace = True)
law.state.replace('47', 47, inplace = True)
#South Atlantic Division
law.state.replace('51', 51, inplace = True)
law.state.replace('52', 52, inplace = True)
law.state.replace('53', 53, inplace = True)
law.state.replace('54', 54, inplace = True)
law.state.replace('55', 55, inplace = True)
law.state.replace('56', 56, inplace = True)
law.state.replace('57', 57, inplace = True)
law.state.replace('58', 58, inplace = True)
law.state.replace('59', 59, inplace = True)
#East South Central Division
law.state.replace('61', 61, inplace = True)
law.state.replace('62', 62, inplace = True)
law.state.replace('63', 63, inplace = True)
law.state.replace('64', 64, inplace = True)
#West South Central Division
law.state.replace('71', 71, inplace = True)
law.state.replace('72', 72, inplace = True)
law.state.replace('73', 73, inplace = True)
law.state.replace('74', 74, inplace = True)
#Mountain Division
law.state.replace('81', 81, inplace = True)
law.state.replace('82', 82, inplace = True)
law.state.replace('83', 83, inplace = True)
law.state.replace('84', 84, inplace = True)
law.state.replace('85', 85, inplace = True)
law.state.replace('86', 86, inplace = True)
law.state.replace('87', 87, inplace = True)
law.state.replace('88', 88, inplace = True)
#Pacific Division
law.state.replace('91', 91, inplace = True)
law.state.replace('92', 92, inplace = True)
law.state.replace('93', 93, inplace = True)
law.state.replace('94', 94, inplace = True)
law.state.replace('95', 95, inplace = True)
#left over state numbers
#putting all numbers 1-9 into their divisions (1st number is the division they are in)
#doesn't matter what the state is because the categories will be their divisions
law.state.replace('7', 71, inplace = True)
law.state.replace('9', 91, inplace = True)
law.state.replace(1, 11, inplace = True)
law.state.replace(2, 21, inplace = True)
law.state.replace(3, 31, inplace = True)
law.state.replace(4, 41, inplace = True)
law.state.replace(5, 51, inplace = True)
law.state.replace(6, 61, inplace = True)
law.state.replace(7, 71, inplace = True)
law.state.replace(8, 81, inplace = True)
law.state.replace(9, 91, inplace = True)

#creating categories for state divisions (dummy variables)    
law['NE_Division'] = law['state'].isin([11,12,13,14,15,16]).astype(int)
law['Middle_Atlantic_Division'] = law['state'].isin([21,22,23]).astype(int)
law['East_North_Central_Division'] = law['state'].isin([31,32,33,34,35]).astype(int)
law['West_North_Central_Division'] = law['state'].isin([41,42,43,44,45,46,47]).astype(int)
law['South_Atlantic_Division'] = law['state'].isin([51,52,53,54,55,56,57,58,59]).astype(int)
law['East_South_Central_Division'] = law['state'].isin([61,62,63,64]).astype(int)
law['West_South_Central_Division'] = law['state'].isin([71,72,73,74]).astype(int)
law['Mountain_Division'] = law['state'].isin([81,82,83,84,85,86,87,88]).astype(int)
law['Pacific_Division'] = law['state'].isin([91,92,93,94,95]).astype(int)

print(law.state.unique())

[63 86 93 84 16 53 59 58 82 33 47 61 52 11 34 41 64 46 88 85 21 44 31 92
 23 15 62 74 13 54 91 35 94 51 95 32 42 72 43 12 22 57 45 87 55 81 73 83
 71]


In [9]:
#fixing race category to be dummy variable
law['race_white'] = (law['race'] == 1).astype(int)
law['race_black'] = (law['race'] == 2).astype(int)
law['race_american_indian'] = (law['race'] == 3).astype(int)
law['race_asian_or_pacific_is'] = (law['race'] == 4).astype(int)
law['race_other'] = (law['race'] == 5).astype(int)
law['race_mixed_black_white'] = (law['race'] == 6).astype(int)
law['race_mixed_white_american_indian'] = (law['race'] == 7).astype(int)
law['race_mixed_white_asian'] = (law['race'] == 8).astype(int)
law['race_mixed_black_american_indian'] = (law['race'] == 10).astype(int)
law['race_mixed_black_asian'] = (law['race'] == 11).astype(int)

In [10]:
#fixing marital variable
law['married'] = law['marital'].isin([1,2,3]).astype(int)
law['single'] = law['marital'].isin([4,5,6,7]).astype(int)

In [11]:
#creating dummy variable for sex (female)
law['female'] = (law["sex"] == 2).astype(int)

In [12]:
#creating dummy variable for grade92
law['grade92'].min() #33, 5th-6th grade (middle school)
law['grade92'].max() #46, doctorate degree
law['grade92'].mean() #about 45, professional school degree (includes JD)
law.loc[law['grade92'] < 45] #only 185 observations with grade92 less than 45
law['ed_low'] = (law['grade92']< 45).astype(int) #less than professional degree (includes bachelors and masters)
law['ed_high'] = (law['grade92']>=45).astype(int) #equal to or higher than professional degree (includes professional degree and doctorate)

In [13]:
#fixing prcitshp variable to be int instead of str
law.prcitshp.replace('Native, Born In US', 1, inplace = True)
law.prcitshp.replace('Foreign Born, US Cit By Naturalization', 1, inplace = True)
law.prcitshp.replace('Foreign Born, Not a US Citizen', 0, inplace = True)
law.prcitshp.replace('Native, Born Abroad Of US Parent(s)', 1, inplace = True)
law.prcitshp.replace('Native, Born in PR or US Outlying Area', 1, inplace = True)

#making prcitshp a dummy variable
law['us_citizen'] = (law['prcitshp'] == 1).astype(int)

In [14]:
#fixing ownchild to be dummy variable
law['no_child'] = (law['ownchild'] == 0).astype(int)
law['one_child'] = (law['ownchild'] == 1).astype(int)
law['two_child'] = (law['ownchild'] == 2).astype(int)
law['three_child'] = (law['ownchild'] == 3).astype(int)
law['four_child'] = (law['ownchild'] == 4).astype(int)
law['five_child'] = (law['ownchild'] == 5).astype(int)
law['six_child'] = (law['ownchild'] == 6).astype(int)

## BIC in full sample
**Aim: good fit and simplicity** (penalizes for model complexity)

In [15]:
reg1 = smf.ols(formula="earnperhr~age+ female", data=law).fit(cov_type="HC1")
#Male is the reference category for female
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:              earnperhr   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     17.32
Date:                Fri, 19 Jan 2024   Prob (F-statistic):           3.92e-08
Time:                        23:55:30   Log-Likelihood:                -4786.5
No. Observations:                1119   AIC:                             9579.
Df Residuals:                    1116   BIC:                             9594.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.9342      2.076     16.350      0.0

In [16]:
reg1.bic

9594.0812746174

In [17]:
reg2 = smf.ols(formula="earnperhr~age+ female+uhourse+ed_high", data=law).fit(cov_type="HC1")
#Male is the reference category for female, ed_low is the reference category for ed_high
print(reg2.summary())

                            OLS Regression Results                            
Dep. Variable:              earnperhr   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     21.70
Date:                Fri, 19 Jan 2024   Prob (F-statistic):           2.94e-17
Time:                        23:55:30   Log-Likelihood:                -4761.9
No. Observations:                1119   AIC:                             9534.
Df Residuals:                    1114   BIC:                             9559.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     31.3915      3.594      8.735      0.0

In [18]:
reg2.bic

9558.988087975542

In [19]:
reg3 = smf.ols(formula='''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                East_South_Central_Division+West_South_Central_Division+
                Mountain_Division+race_black+race_american_indian+
                race_asian_or_pacific_is+race_other+race_mixed_black_white+
                race_mixed_white_american_indian+race_mixed_white_asian+
                race_mixed_black_american_indian+race_mixed_black_asian+us_citizen''', data=law).fit(cov_type="HC1")
#Male is the reference category for female, ed_low is the reference category for ed_high
#Pacific_Division is the reference category for all listed state divisions
#race_white is the reference category for all listed races
#Not being a US citizen is the reference category for us_citizen
print(reg3.summary())

                            OLS Regression Results                            
Dep. Variable:              earnperhr   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     107.3
Date:                Fri, 19 Jan 2024   Prob (F-statistic):          3.05e-234
Time:                        23:55:30   Log-Likelihood:                -4741.4
No. Observations:                1119   AIC:                             9529.
Df Residuals:                    1096   BIC:                             9644.
Df Model:                          22                                         
Covariance Type:                  HC1                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [20]:
reg3.bic
#strong multicollinearity could be due to the fact that many of the observations have race of white.
#there are only 199 with other races specified

9644.230054440795

In [21]:
reg4 = smf.ols(formula='''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                East_South_Central_Division+West_South_Central_Division+
                Mountain_Division+race_black+race_american_indian+
                race_asian_or_pacific_is+race_other+race_mixed_black_white+
                race_mixed_white_american_indian+race_mixed_white_asian+
                race_mixed_black_american_indian+race_mixed_black_asian+us_citizen
                +married+one_child+two_child+three_child+
                four_child+five_child+six_child''', data=law).fit(cov_type="HC1")
#Male is the reference category for female, ed_low is the reference category for ed_high
#Pacific_Division is the reference category for all listed state divisions
#race_white is the reference category for all listed races
#Not being a US citizen is the reference category for us_citizen
#single is the reference category for married
#no_child is the reference category for those with 1-6 children
print(reg4.summary())

                            OLS Regression Results                            
Dep. Variable:              earnperhr   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     94.84
Date:                Fri, 19 Jan 2024   Prob (F-statistic):          6.31e-247
Time:                        23:55:30   Log-Likelihood:                -4734.3
No. Observations:                1119   AIC:                             9527.
Df Residuals:                    1090   BIC:                             9672.
Df Model:                          28                                         
Covariance Type:                  HC1                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [22]:
reg4.bic

9672.228139011642

In [23]:
#reordering covariate order for Stargazer
star = Stargazer([reg1, reg2, reg3, reg4])
cov_order = ['Intercept', 'age', 'female', 'uhourse', 'ed_high', 'NE_Division', 'Middle_Atlantic_Division',
                          'East_North_Central_Division', 'West_North_Central_Division', 'South_Atlantic_Division',
                          'East_South_Central_Division', 'West_South_Central_Division', 'Mountain_Division',
                          'race_black','race_american_indian', 'race_asian_or_pacific_is', 'race_other', 'race_mixed_black_white',
                          'race_mixed_white_american_indian','race_mixed_white_asian', 'race_mixed_black_american_indian',
                          'race_mixed_black_asian', 'us_citizen', 'married', 'one_child', 'two_child', 'three_child',
                          'four_child', 'five_child', 'six_child']
star.covariate_order(cov_order)
star

In [24]:
#As to be expected, many of the variables are not statistically significant.

In [25]:
reg_names = ['reg1', 'reg2', 'reg3', 'reg4']
reg_bics = [reg1.bic, reg2.bic, reg3.bic, reg4.bic]
regs_and_bics = list(zip(reg_names,reg_bics))

#shows which regression had the lowest/most favorable bic = Regression 2
sorted(regs_and_bics, key = lambda x: x[1])

#Considering the R-squared values (on the lower end) as well as the BIC values, despite regression 2's model seeming to be a more
#favorable model to use when it comes to this particularly prediction, it is not necessarily a good model in
#general. It is the best out of a bad bunch of 4 regressions/models.

[('reg2', 9558.988087975542),
 ('reg1', 9594.0812746174),
 ('reg3', 9644.230054440795),
 ('reg4', 9672.228139011642)]

## Linear Regression with k-fold cross-validation (cross-validated RMSE)
**Aim: See how well the models predict outcomes across different subsets of manufactured 'original' and 'live' data (how accurate the predictions are). Helps to avoid over- and underfitting data**

In [26]:
#creating 5 splits. 80% in training set, 20% in test set
from sklearn.model_selection import KFold
k = KFold(n_splits=5, shuffle=False, random_state=None)

In [27]:
for train_index, test_index in k.split(law):
    print(train_index, '\n', '\n', test_index, '\n')

print(f"Length of training_sets: {len(train_index)}")
print(f"Length of test_sets: {len(test_index)}")

[ 224  225  226  227  228  229  230  231  232  233  234  235  236  237
  238  239  240  241  242  243  244  245  246  247  248  249  250  251
  252  253  254  255  256  257  258  259  260  261  262  263  264  265
  266  267  268  269  270  271  272  273  274  275  276  277  278  279
  280  281  282  283  284  285  286  287  288  289  290  291  292  293
  294  295  296  297  298  299  300  301  302  303  304  305  306  307
  308  309  310  311  312  313  314  315  316  317  318  319  320  321
  322  323  324  325  326  327  328  329  330  331  332  333  334  335
  336  337  338  339  340  341  342  343  344  345  346  347  348  349
  350  351  352  353  354  355  356  357  358  359  360  361  362  363
  364  365  366  367  368  369  370  371  372  373  374  375  376  377
  378  379  380  381  382  383  384  385  386  387  388  389  390  391
  392  393  394  395  396  397  398  399  400  401  402  403  404  405
  406  407  408  409  410  411  412  413  414  415  416  417  418  419
  420 

In [28]:
### Cross validate OLS with combining sklearn k-fold cross validation and statsmodels ols formula


def cv_reg(formula, data, kfold, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []

    # Calculating OLS for each fold

    for train_index, test_index in k.split(data):
        # print("TRAIN:", train_index, "TEST:", test_index)
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]
        rmse_list += [rmse(data_train[formula.split("~")[0]], model.predict(data_train))]

    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
    }


def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cv_list))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result

In [29]:
#putting each regression/model into the function
cv1 = cv_reg("earnperhr~age+female", law, k, "HC1")

cv2 = cv_reg("earnperhr~age+ female+uhourse+ed_high", law, k, "HC1")

cv3 = cv_reg('''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                 East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                 East_South_Central_Division+West_South_Central_Division+
                 Mountain_Division+race_black+race_american_indian+
                 race_asian_or_pacific_is+race_other+race_mixed_black_white+
                 race_mixed_white_american_indian+race_mixed_white_asian+
                 race_mixed_black_american_indian+race_mixed_black_asian+us_citizen''', law, k, "HC1")

cv4 = cv_reg('''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                East_South_Central_Division+West_South_Central_Division+
                Mountain_Division+race_black+race_american_indian+
                race_asian_or_pacific_is+race_other+race_mixed_black_white+
                race_mixed_white_american_indian+race_mixed_white_asian+
                race_mixed_black_american_indian+race_mixed_black_asian+us_citizen
                +married+one_child+two_child+three_child+
                four_child+five_child+six_child''', law, k, "HC1")

cv_list = [cv1, cv2, cv3, cv4]

In [30]:
summarize_cv(cv_list)
#RMSE for every single fold and every model
#lowest avg RMSE is on most complicated model (Model4)!!
#this means that on average, model 4's predictions deviate from the actual values (earnings per hour) by about
#16.5 USD. 

,Model1,Model2,Model3,Model4
Fold1,17.518910,17.139792,16.784541,16.681709
Fold2,17.279032,16.862471,16.604411,16.523700
Fold3,17.477462,17.098445,16.665863,16.593329
Fold4,17.238952,16.830971,16.495393,16.278412
Fold5,17.630345,17.311502,17.012706,16.897187
Average,17.428940,17.048636,16.712583,16.594867


## RMSE on full sample
**Aim: Assess how well models perform on whole dataset- to see how accurate predictions are**

In [31]:
#RMSE on full sample (reg1)

formula="earnperhr~age+ female"

full_model = smf.ols(formula, data=law).fit(cov_type="HC1")

# Make predictions on the full dataset
law['predictions'] = full_model.predict(law)

# Calculate RMSE
rmse_full = np.sqrt(mean_squared_error(law['earnperhr'], law['predictions']))

print(f"RMSE on 1st regression model (full sample): {rmse_full}\n")
print(law.earnperhr.describe())
#means that on average, the model's predictions deviate from the actual values (earnings per hour) by about 17.43 USD.
#this is large considering 75% of the observations have employees making under 53 USD per hour.

RMSE on 1st regression model (full sample): 17.436236706187387

count    1119.000000
mean       41.119032
std        17.748647
min         0.000000
25%        26.602533
50%        40.865250
75%        53.840000
max       128.204667
Name: earnperhr, dtype: float64


In [32]:
#RMSE on full sample (reg2)

formula="earnperhr~age+ female+uhourse+ed_high"

full_model = smf.ols(formula, data=law).fit(cov_type="HC1")

# Make predictions on the full dataset
law['predictions'] = full_model.predict(law)

# Calculate RMSE
rmse_full = np.sqrt(mean_squared_error(law['earnperhr'], law['predictions']))

print(f"RMSE on 2nd regression model (full sample): {rmse_full}")
#means that on average, the model's predictions deviate from the actual values (earnings per hour) by about 17.05 USD.
#this is large considering 75% of the observations have employees making under 53 USD per hour.

RMSE on 2nd regression model (full sample): 17.05760899271748


In [33]:
#RMSE on full sample (reg3)

formula='''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                East_South_Central_Division+West_South_Central_Division+
                Mountain_Division+race_black+race_american_indian+
                race_asian_or_pacific_is+race_other+race_mixed_black_white+
                race_mixed_white_american_indian+race_mixed_white_asian+
                race_mixed_black_american_indian+race_mixed_black_asian+us_citizen'''

full_model = smf.ols(formula, data=law).fit(cov_type="HC1")

# Make predictions on the full dataset
law['predictions'] = full_model.predict(law)

# Calculate RMSE
rmse_full = np.sqrt(mean_squared_error(law['earnperhr'], law['predictions']))

print(f"RMSE on 3rd regression model (full sample): {rmse_full}")
#means that on average, the model's predictions deviate from the actual values (earnings per hour) by about 16.74 USD.
#this is large considering 75% of the observations have employees making under 53 USD per hour.

RMSE on 3rd regression model (full sample): 16.747050888716466


In [34]:
#RMSE on full sample (reg4)

formula='''earnperhr~age+ female+uhourse+ed_high+NE_Division+Middle_Atlantic_Division+ 
                East_North_Central_Division+West_North_Central_Division+South_Atlantic_Division+
                East_South_Central_Division+West_South_Central_Division+
                Mountain_Division+race_black+race_american_indian+
                race_asian_or_pacific_is+race_other+race_mixed_black_white+
                race_mixed_white_american_indian+race_mixed_white_asian+
                race_mixed_black_american_indian+race_mixed_black_asian+us_citizen
                +married+one_child+two_child+three_child+
                four_child+five_child+six_child'''

full_model = smf.ols(formula, data=law).fit(cov_type="HC1")

# Make predictions on the full dataset
law['predictions'] = full_model.predict(law)

# Calculate RMSE
rmse_full = np.sqrt(mean_squared_error(law['earnperhr'], law['predictions']))

print(f"RMSE on 4th regression model (full sample): {rmse_full}")
#means that on average, the model's predictions deviate from the actual values (earnings per hour) by about 16.64 USD.
#this is somewhat considering 75% of the observations have employees making under 53 USD per hour.

RMSE on 4th regression model (full sample): 16.64170018292906


In [35]:
#The best RMSE in the full sample was from model 4/regression 4. The predicted values deviate 
#on average from the actual values by 16.64 USD in earnings per hour. 

## Comparison of Measures of Fit

In [36]:
#Table Showing comparison of the measure of fit values
measure_fit_table = pd.DataFrame({
    "Full Sample BIC":{'Model 1': 9594.08, 'Model 2': 9558.98, 'Model 3': 9644.23, 'Model 4':9672.22}, 
    "Avg Cross-Validated RMSE": {'Model 1': 17.42, 'Model 2': 17.04, 'Model 3': 16.71, 'Model 4':16.59}, 
    "Full Sample RMSE": {'Model 1': 17.43, 'Model 2': 17.05, 'Model 3': 16.74, 'Model 4':16.64}
}).T
measure_fit_table 

,Model 1,Model 2,Model 3,Model 4
Full Sample BIC,9594.08,9558.98,9644.23,9672.22
Avg Cross-Validated RMSE,17.42,17.04,16.71,16.59
Full Sample RMSE,17.43,17.05,16.74,16.64


BIC: When penalized for complexity (using BIC), naturally Models 1 and especially model 2 performed better than 3 and 4 as they had lower BIC scores. Despite this, their R-squared measures were low in general; perhaps a sign that the models don't fit the data well in general.

RMSE: However, when not penalized for complexity (both instances of RMSE), Model 4 outperforms the others in both cross-validated RMSE and RMSE taken from the full sample. 

## Prediction
It should be noted that I am using the linear regressions/models from the full sample to predict the target value. 

In [37]:
#variables for prediction

data = law[
    [
        'age', 'female','uhourse','ed_high','NE_Division','Middle_Atlantic_Division', 
                'East_North_Central_Division','West_North_Central_Division','South_Atlantic_Division',
                'East_South_Central_Division','West_South_Central_Division',
                'Mountain_Division','race_black','race_american_indian',
                'race_asian_or_pacific_is','race_other','race_mixed_black_white',
                'race_mixed_white_american_indian','race_mixed_white_asian',
                'race_mixed_black_american_indian','race_mixed_black_asian','us_citizen',
                'married','one_child','two_child','three_child',
                'four_child','five_child','six_child'
    ]
]

In [38]:
#checking data types to create target values for prediction
data.dtypes

age                                   int64
female                                int64
uhourse                             float64
ed_high                               int64
NE_Division                           int64
Middle_Atlantic_Division              int64
East_North_Central_Division           int64
West_North_Central_Division           int64
South_Atlantic_Division               int64
East_South_Central_Division           int64
West_South_Central_Division           int64
Mountain_Division                     int64
race_black                            int64
race_american_indian                  int64
race_asian_or_pacific_is              int64
race_other                            int64
race_mixed_black_white                int64
race_mixed_white_american_indian      int64
race_mixed_white_asian                int64
race_mixed_black_american_indian      int64
race_mixed_black_asian                int64
us_citizen                            int64
married                         

In [39]:
#dataframe of target values
#Target to predict: A legal employee who is 40 years old, female, highly educated, 
#lives in the Mountain Division of the United States, race is Black, is a U.S. citizen, is single, 
#and has 2 children

new = pd.DataFrame(pd.Series({
    'age':40, 'female':1,'uhourse':40,'ed_high':1,'NE_Division':0,'Middle_Atlantic_Division':0, 
                'East_North_Central_Division':0,'West_North_Central_Division':0,'South_Atlantic_Division':0,
                'East_South_Central_Division':0,'West_South_Central_Division':0,
                'Mountain_Division':1,'race_black':1,'race_american_indian':0,
                'race_asian_or_pacific_is':0,'race_other':0,'race_mixed_black_white':0,
                'race_mixed_white_american_indian':0,'race_mixed_white_asian':0,
                'race_mixed_black_american_indian':0,'race_mixed_black_asian':0,'us_citizen':1,
                'married':0,'one_child':0,'two_child':1,'three_child':0,
                'four_child':0,'five_child':0,'six_child':0, 'earnperhr':np.nan
})).T
new

,age,female,uhourse,ed_high,NE_Division,Middle_Atlantic_Division,East_North_Central_Division,West_North_Central_Division,South_Atlantic_Division,East_South_Central_Division,...,race_mixed_black_asian,us_citizen,married,one_child,two_child,three_child,four_child,five_child,six_child,earnperhr
0,40.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN


In [40]:
#turning off scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [41]:
#description of reg1 residuals
reg1.resid.describe()

count   1119.000
mean      -0.000
std       17.444
min      -46.947
25%      -13.484
50%        0.003
75%       12.626
max       88.383
dtype: float64

In [42]:
#df showing reg1 prediction values
p1=reg1.get_prediction(new).summary_frame()
p1

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,37.717,0.861,36.028,39.405,3.455,71.979


In [43]:
#description of reg2 residuals
(reg2.fittedvalues-law.earnperhr).describe()

count   1119.000
mean      -0.000
std       17.065
min      -82.291
25%      -12.421
50%       -0.474
75%       12.988
max       51.838
dtype: float64

In [44]:
#df showing reg2 prediction values
p2=reg2.get_prediction(new).summary_frame()
p2

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,40.037,1.000,38.078,41.997,6.473,73.602


In [45]:
#description of reg3 residuals
(reg3.fittedvalues-law.earnperhr).describe()

count   1119.000
mean      -0.000
std       16.755
min      -77.620
25%      -11.686
50%       -0.000
75%       12.476
max       56.188
dtype: float64

In [46]:
#df showing reg3 prediction values
p3=reg3.get_prediction(new).summary_frame()
p3

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,40.475,3.477,33.659,47.290,6.615,74.334


In [47]:
#description of reg4 residuals
(reg4.fittedvalues-law.earnperhr).describe()

count   1119.000
mean      -0.000
std       16.649
min      -75.626
25%      -11.827
50%       -0.275
75%       12.095
max       55.594
dtype: float64

In [48]:
#df showing reg4 prediction values
p4=reg4.get_prediction(new).summary_frame()
p4

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,43.059,3.754,35.701,50.417,9.202,76.916


In [49]:
# summary of predictions and PI 95% version
pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(95%)", "PI_high(95%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model2": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p3[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model4": p4[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0]
    }
).set_index(" ")

,Model1,Model2,Model3,Model4
,,,,
Predicted,37.717,40.037,40.475,43.059
PI_low(95%),3.455,6.473,6.615,9.202
PI_high(95%),71.979,73.602,74.334,76.916


In [50]:
#Differences between upper and lower prediction intervals for 95% confidence interval
print(p1.obs_ci_upper - p1.obs_ci_lower)
print(p2.obs_ci_upper - p2.obs_ci_lower)
print(p3.obs_ci_upper - p3.obs_ci_lower)
print(p4.obs_ci_upper - p4.obs_ci_lower)

#Model 2 has the narrowest prediction interval (Model 2 is 95% confident that the predicted earnings per hour 
#for the specified target fall within 6.47 USD and 73.60 USD. This is a wide prediction interval; however, 
#it should be noted that prediction intevals are wider than CIs because they predict future observations, 
#in which there is more uncertainty)

0   68.524
dtype: float64
0   67.129
dtype: float64
0   67.719
dtype: float64
0   67.715
dtype: float64


In [51]:
# summary of predictions and PI 80% version
p1=reg1.get_prediction(new).summary_frame(alpha=0.2)
p2=reg2.get_prediction(new).summary_frame(alpha=0.2)
p3=reg3.get_prediction(new).summary_frame(alpha=0.2)
p4=reg4.get_prediction(new).summary_frame(alpha=0.2)

pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(80%)", "PI_high(80%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model2": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p3[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model4": p4[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0]
    }
).set_index(" ")

,Model1,Model2,Model3,Model4
,,,,
Predicted,37.717,40.037,40.475,43.059
PI_low(80%),15.314,18.091,18.335,20.921
PI_high(80%),60.119,61.984,62.614,65.197


In [52]:
##Differences between upper and lower prediction intervals for 80% confidence interval
print(p1.obs_ci_upper - p1.obs_ci_lower)
print(p2.obs_ci_upper - p2.obs_ci_lower)
print(p3.obs_ci_upper - p3.obs_ci_lower)
print(p4.obs_ci_upper - p4.obs_ci_lower)

#Model 2 again has the narrowest prediction interval (Model 2 is 80% confident that the predicted earnings per hour 
#for the specified target fall within 18.09 USD and 61.98 USD. This is a wide prediction interval; however, 
#it should be noted that prediction intevals are wider than CIs because they predict future observations, 
#in which there is more uncertainty)

0   44.805
dtype: float64
0   43.893
dtype: float64
0   44.279
dtype: float64
0   44.276
dtype: float64


## Summary

There is a clear trade off between model complexity and predictive accuracy seen
in the comparisons between BIC and RMSE. BIC is more concerned about penalizing model complexity- therefore, 
providing a model with good fit and simplicity. It therefore also protects against overfitting the original data. RMSE with cross-validation also helps protect against the possibility of over- or underfitting the data; however overall, RMSE is not concerned with penalizing models based on their complexity and instead focuses on predicting accurate targets in the live data. This trade off is important to consider when deciding which model to use in order to predict the outcome variable in the future live data. 
PIs for predicting individual predictions based on the target values specified go even further in providing ranges for each model in which future observations might fall. That being said, while the aim of PIs is to account for uncertainty, as I applied them above, they are another good way to compare quality between models.

When choosing which model to pick to best predict live data, these trade offs must be accounted for. Using my models, if one cared more about the possibility of overfitting the data and therefore wanted to penalize for model complexity, they might choose Model 1 or 2 (lowest BIC values) to predict the live data due to their simplicity in comparison to Models 3 and 4. However, if more concerned with accurate predictions of the live data target values, one might choose Model 4 (lowest MSE and RMSE values). Personally, as the models do not seem to fit the original data very well anyway due to the low R-squared values, I think there is very little risk in overfitting the original data. Therefore, I might use the average K-Fold cross-validation RMSE values and go with Model 4 as my model of choice when predicting the live data to focus more on predicting accurate target values.